# Black positivity
### News article scraping

Notebook for collecting Black news articles from positive good black news and connecting to newsapi

# Import

In [3]:
#Import and remember to have your news api key set as en environment variable

%load_ext dotenv
%dotenv C:/PATH/TO/API/KEY.env
    
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pprint import pprint as pp
from nytimesarticle import articleAPI
from bs4 import BeautifulSoup
from functools import reduce
from newsapi import NewsApiClient
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import pandas as pd
import numpy as np
import threading
import sqlite3
import time
import re 
import requests
import matplotlib.pyplot
import psycopg2
import os
from dotenv import load_dotenv

In [4]:
#Loading api key and setting to variable 'newsapi'
load_dotenv()
api_key = os.getenv('API_KEY')
newsapi = NewsApiClient(api_key=  api_key)

# Function definitions

In [18]:
def article_grab(url,name ):
    """ Function to take a url form goodblack news and return a dataframe of headlines"""
    wd = webdriver.Chrome()
    time.sleep(5)
    wd.get(url)
    while True:
        try:
            python_button = wd.find_elements_by_xpath('//*[@id="infinite-handle"]/span/button')[0]
            time.sleep(2)
            python_button.click()
            time.sleep(5)
        except Exception as e:
            print(e)
            break
    
    print("Complete")
    
    
    time.sleep(10)
    
    ids = wd.find_elements_by_xpath('//*[contains(@id,"post-")]')
    
    print('got elements')

    comment_ids = []
    
    for i in ids:
        
        comment_ids.append(i.get_attribute('id'))
    
    regex = re.compile('.*like')
    
    filtered = [i for i in comment_ids if not regex.search(i)]
    
    df = pd.DataFrame(columns = ['user_id'])####
    
    print('gotdf')
    
    for title in filtered:
        try:
            userid_element = wd.find_elements_by_xpath('//*[@id="' + title + '"]/header/div/div/a/h1')[0]
            time.sleep(5)
            userid = userid_element.text
            df.loc[len(df)] = [userid]
            print(userid)
        except:
            print('dfadded')
            continue
    print('close')
    wd.close()
    df.to_csv(str(name) + '.csv')
    print('DONE')

In [148]:
def conv_newapi(result):
    article_dict={
    'source':[],
    'title':[]#,
#    'category':[]
    }

    for i in range(len(result['articles'])):
    
        article_dict['source'].append(result['articles'][i]['source']['name'])
        article_dict['title'].append(result['articles'][i]['title'])
 #       article_dict['category'] = cat_list.values
        global articledf    
        articledf = pd.DataFrame(article_dict)
    return articledf
def get_articles(cat, query):
    
    catdict = newsapi.get_sources(category = cat)
    sources = catdict['sources']
    source_list = []
    #cat_list = list([cat] * 100)
    #print(cat_list)
    df_list = []
                                      
    for example in sources:
        if example['language'] == 'en':
            source_list.append(example['id'])
        else:
            pass
                                
    for source in source_list:
        cat_article = newsapi.get_everything(q=query,
                            page_size = 100,
                            sort_by='relevancy',
                            sources = source,
                            language='en')
#         n_results = len(cat_article['articles'])
#         cat_list = list([cat] * n_results)
#         print(n_results)
#         cat_list = pd.Series(cat_list)
#         print(cat_list)
#         print(len(cat_list))
        articledf = conv_newapi(cat_article)#, cat_list)
        df_list.append(articledf)
        df = pd.concat(df_list)
        time.sleep(10)
    return df
        
                                  

# Collection

goodblacknews scraping

In [20]:


url1 = 'https://goodblacknews.org/category/businessfinance/'
url2 = 'https://goodblacknews.org/category/arts-style/'
url3 = 'https://goodblacknews.org/category/entertainment-2/mediainternet/'
url4 = 'https://goodblacknews.org/category/sports/'
url5 = 'https://goodblacknews.org/category/lifestyle-2/'
url6 = 'https://goodblacknews.org/category/books/'
url7 = 'https://goodblacknews.org/category/education/'
url8 = 'https://goodblacknews.org/category/technology/'
url9 = 'https://goodblacknews.org/category/news/u-s/'
url10 = 'https://goodblacknews.org/category/commemorations/'
url11 = 'https://goodblacknews.org/category/international/'
url12 = 'https://goodblacknews.org/category/politics/'

article_grab(url1, 'biz')
article_grab(url2, 'art')
article_grab(url3, 'ent')
article_grab(url4, 'sports')
article_grab(url5, 'life')
article_grab(url6, 'books')
article_grab(url7, 'ed')
article_grab(url8, 'tech')
article_grab(url9, 'us')
article_grab(url10, 'commemorations')
article_grab(url11,'int')
article_grab(url12,'pol')

Now to get news headlines from other sources using news api

In [205]:
#Collecting articles from five categories using three search terms:
bla_sports = get_articles('sports', 'black american')
aa_sports = get_articles('sports', 'african american')
black_americans_sports = get_articles('sports', 'black americans')
bla_general = get_articles('general', 'black american')
aa_gen = get_articles('general', 'african american')
black_a_gen = get_articles('general', 'black americans')
bl_biz = get_articles('business', 'black american')
aa_biz = get_articles('business', 'black american')
black_a_biz = get_articles('business', 'black american')
bl_tech = get_articles('technology', 'black american')
aa_tech = get_articles('technology', 'african american')
black_aatech = get_articles('technology', 'black americans')
bl_ent = get_articles('entertainment', 'black american')
aa_ent = get_articles('entertainment', 'african american')
black_ent = get_articles('entertainment', 'black americans')



In [209]:
#Now concat and save as csv files

blsportsdf = pd.concat(bla_sports, aa_sports, black_americans_sports)
blgendf = pd.concat(black_a_gen, aa_gen, bla_general)
blbizdf = pd.concat(bl_biz,aa_biz, black_a_biz)
blsportsdf.to_csv('blsports.csv')
blgendf.to_csv('blgen.csv')
blbizdf.to_csv('blbiz.csv')
tech = [bl_tech, aa_tech, black_aatech]
bltech = pd.concat(tech)
bltech.to_csv('bltech.csv')
blent = pd.concat([bl_ent,aa_ent,black_ent])
blent.to_csv('blent.csv')